# Press "SHIFT + ENTER" to run a cell, all cells should be run in order, from top to bottom

### First we import some libraries

In [2]:
import numpy as np
from PIL import Image

### Then we set some parameters:
1. Change 4096 to the size you want your unsig to be (width and height will be the same)

In [3]:
#pixel dimension
dim = 4096

## input your unsigs:
1. index (number)
2. number of properties
3. the values of these properties (be careful, words like 'Green' and 'Normal' need to be enclosed in quotes

In [8]:
#replace the content inside {} with your unsig's properties
unsig = {'index': 2240,
         'num_props': 3,
         'properties': {
             'multipliers'   : [2, 2, 4],
             'colors'        : ['Green', 'Blue', 'Red'],
             'distributions' : ['CDF', 'CDF', 'CDF'],
             'rotations'     : [0, 0, 90]}}

## Run the cell below!

In [61]:
# Daniel:
#   Generates a bell curve (normal distribution)
def norm(x , mean , std):
    p = (np.pi*std) * np.exp(-0.5*((x-mean)/std)**2)
    return p

def scale_make2d(s):
    # Sources:
    #   https://numpy.org/doc/stable/reference/generated/numpy.interp.html
    #   Returns the one-dimensional piecewise linear interpolant to a function with given discrete data points (xp, fp), evaluated at x.
    #   https://en.wikipedia.org/wiki/Linear_interpolation
    # Notes:
    #   s: coordinates where to evaluate the interpolated values
    #   (s.min(), s.max()): 1-D floats of the X plane, in order from min to max
    #   (0, u_range):  1-D floats of the Y plane, same size as the previous one
    # Daniel:
    #   aka: joins dots with lines
    #   smooths input s into dimension "dim"
    scaled = np.interp(s, (s.min(), s.max()), (0, u_range))
    # Note:
    #   Construct an array by repeating (dim, 1) the number of times given by "scaled".
    # Daniel:
    #   Copies the one array into the other dimension
    two_d = np.tile(scaled, (dim, 1))
    return two_d

# here "nft" seems unused. Previously gen_nft(_nft)
# Daniel:
#   why is it unused? -- Alexander is taking advantage of the global scope.
#   Makes the code smaller, although it's not generally recommended.
def gen_nft():
    idx = unsig['index']
    props = unsig['properties']
    
    # Daniel:
    #   Fills the area with zeroes, aka black
    n = np.zeros((dim, dim, 3)).astype(np.uint32)

    # Daniel:
    #   per property in an unsig...
    #   rotates and multiplies the distribution of that property (Normal or CDF)
    #   then sets the color on each property of the resulting array.
    for i in range(unsig['num_props']):
        mult = props['multipliers'][i]
        col = props['colors'][i]
        dist = props['distributions'][i]
        rot = props['rotations'][i]
        c = channels[col]
        # this is the source of the rollover
        buffer =  mult * np.rot90(dists[dist], k=(rot / 90))
        n[ :, :, c ] = n[ :, :, c ] + buffer

    # Daniel:
    #   this is like, compressing the prior values into the 0 to 255 range.
    #   Here is where the rollover happens: n is uint32 and here we force it down to uint8.
    #   Q: Can the rest be coded on uint16 and then here compressed to uint8?
    #   A: Perhaps, but many of the constants need to change.
    n = np.interp(n, (0, u_range), (0, 255)).astype(np.uint8)

    return (idx, n)

if __name__ == '__main__':
    # What this does:
    # - Creates the empty space to fill "dim" in 2d
    # - Creates a normal distribution (used for "Normal" properties in unsigs).
    # - Creates a cumulative distribution (the S shape of unsigs used on "CDF" properties).
    # - These two are simply 1d arrays, so then Alexander makes them 2d arrays.
    # - then calls gen_nft, which:
    #   - goes over each property, adding a layer resulting of the multiplication and rotation of the distribution of that property
    #   - Adds the color of the layer
    #   - Maps the values into 0-255
    #   - Thus generating arrays of (r, g, b, "Color name") elements
    #   - Which is the exact shape of an array of RGB pixels.
    # - then prints the image

    #setup
    x = list(range(dim))
    u_range = 4294967293
    mean = np.mean(x)

    # Me:
    #   This seems very arbitrary. Probably Alexander's personal touch.
    # Alexander:
    #   Affects the tightness or fatness of the bell curve
    std = dim/6

    # probability and cumulative distribution
    # Daniel:
    #   This is really what determines the shape of unsigs,
    #   It's like a warped map that ensures things fall into place quite nicely
    # Alexander:
    #   The first one makes a 1 dimensional array with a normal distribution.
    #   The second one cumulatively sums the array to make the S curve.
    #   Thanks to the second one, it slowly takes off, speeds up, and then decelerates again
    p_1d = np.array(norm(x, mean, std)).astype(np.uint32)
    c_1d = np.cumsum(p_1d)

    #2d arrays
    p_2d = scale_make2d(p_1d)
    c_2d = scale_make2d(c_1d)

    #dicts for retrieving values
    dists = {'Normal': p_2d, 'CDF': c_2d}
    channels = {'Red': 0, 'Green': 1, 'Blue': 2}
    
    #make your nft
    # unsig here is undefined
    i, nft = gen_nft() # previously: gen_nft(unsig)
    
    img = Image.fromarray(nft)
    img.save(f'unsig_{i:05d}.png')

# <- Look over there, you should see a file named "unsig_YOURUNSIG#.png" right click it and choose download!